In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
# check if data entered properly; column names, missing data etc
#'purchase_data.columns
#'purchase_data.head()
#'purchase_data.dropna(how="any")
#'purchase_data.describe()

In [2]:
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#Player Count
#length of unique values in SN
players = len(purchase_data["SN"].unique())

#Create data frame to hold number of players
players_df= pd.DataFrame({"Total Players": [players]})
players_df

,Total Players
0,576


In [4]:
# create function to format the values (This only worked with the series)
def format(x):
        return "${:.2f}".format(x)

In [5]:
#Purchasing Analysis (Total)
#variables to hold information from purchase data df
unique_items = purchase_data["Item ID"].nunique()
price_avg = format(purchase_data["Price"].mean())
purchases = len(purchase_data["Purchase ID"].unique())
revenue = format(purchase_data["Price"].sum())

#make summary df to hold purchase analysis variables
purchase_analysis=pd.DataFrame({"Number of Unique Items":[unique_items],
                        "Average Price": [price_avg],
                        "Total Purchases":[purchases],
                        "Total Revenue":[revenue]})
purchase_analysis

,Number of Unique Items,Average Price,Total Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [6]:
#Gender Demographics

#group df by gender then count the unique players for each group
gender_group = purchase_data.groupby(['Gender'])
count_gender = gender_group['SN'].nunique()

In [7]:
#variables to hold info for each gender (slice into the count gender)
boy_count= count_gender["Male"]
boy_pct= boy_count/players

women_count = count_gender["Female"]
women_pct= women_count/players

other_count=count_gender["Other / Non-Disclosed"]
other_pct= other_count/players

In [8]:
#summary df to hold variables, format percentages
gender_summary= pd.DataFrame({"Total Count":[boy_count,women_count,other_count],
                             "Percentage of Players": ["{0:.2f}%".format(boy_pct * 100),"{0:.2f}%".format(women_pct * 100),"{0:.2f}%".format(other_pct * 100)]})
#set index labels
gender_summary.set_index(pd.Index(["Male", "Female", "Other"]))

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other,11,1.91%


In [9]:
#Purchasing Analysis (Gender)
#Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender
#run calculations for each gender (alternatively could do group by)
#FEMALES
only_females_df = purchase_data.loc[(purchase_data["Gender"] == "Female"), :]
avg_fem_price = format(only_females_df["Price"].mean())
fem_purchasecount = only_females_df["Purchase ID"].count()
fem_revenue = only_females_df["Price"].sum()
fem_per_person = fem_revenue/women_count

#BOYS
only_boys_df = purchase_data.loc[(purchase_data["Gender"] == "Male"), :]
avg_boy_price = format(only_boys_df["Price"].mean())
boy_purchasecount = only_boys_df["Purchase ID"].count()
boy_revenue = only_boys_df["Price"].sum()
boy_per_person = boy_revenue/boy_count

#OTHER
only_other_df = purchase_data.loc[(purchase_data["Gender"] == "Other / Non-Disclosed"), :]
avg_other_price = format(only_other_df["Price"].mean())
other_purchasecount = only_other_df["Purchase ID"].count()
other_revenue = only_other_df["Price"].sum()
other_per_person = other_revenue/other_count

In [10]:
#data frame with variables created above (format to $)
gpurchase_summary=pd.DataFrame({"Purchase Count":[fem_purchasecount,boy_purchasecount,other_purchasecount],
                                "Average Purchase Price":[avg_fem_price,avg_boy_price,avg_other_price],
                                "Total Purchase Value":[format(fem_revenue),format(boy_revenue),format(other_revenue)],
                                "Avg Total Purchase per Person":[format(fem_per_person),format(boy_per_person),format(other_per_person)]
                                                  })

gender_purchase_summary = gpurchase_summary.set_index(pd.Index(['Female','Male','Other / Non-Disclosed']))
gender_purchase_summary.index.name="Gender"
gender_purchase_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [11]:
#Alternative method of creating table above (with groupby- more efficient)
purchase_countgrp=gender_group['Purchase ID'].count()

price_grouped=gender_group['Price'].mean()
price_grouped_f=price_grouped.map("${:,.2f}".format)


total_purchvalgrp=gender_group['Price'].sum()
total_purchvalgrp_f=total_purchvalgrp.map("${:,.2f}".format)

avgpurchase_grp=gender_group['Price'].sum()/gender_group['SN'].nunique()
avgpurchase_grp_f=avgpurchase_grp.map("${:,.2f}".format)

alt_gender_purchase_summary=pd.DataFrame({"Purchase Count":purchase_countgrp,
                                "Average Purchase Price":price_grouped_f,
                                "Total Purchase Value":total_purchvalgrp_f,
                                "Avg Total Purchase per Person":avgpurchase_grp_f
                                                  })
alt_gender_purchase_summary

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [12]:
#Age Demographics

#Establish bins for ages
bin= [0,9,14,19,24,29,34,39,45]
bin_ages=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

In [13]:
#Age Demographics
#Create dataframe that holds info from each client (without duplicates)
unique_SN=purchase_data.drop_duplicates(subset="SN")

#Categorize the existing players using the age bins- create new column [Age group] to hold the binned ages
unique_SN["Age Group"]=pd.cut(unique_SN["Age"],bin,labels=bin_ages)

#Create new data frame to group by age group bins
age_group=unique_SN.groupby("Age Group")


/Users/leah/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
#series to hold count in each age group 
age_count=age_group['SN'].count()

# calculate percentage for each bin in 'Age Group' 
prct_age_group = (age_group['SN'].count()/players)*100
prct_age_group_f=prct_age_group.map("{:,.2f}%".format)

In [15]:
#Age Demographics Summary
age_demographics = pd.DataFrame({"Total Count":age_count,
                                "Percentage of Players":prct_age_group_f})
age_demographics

,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [16]:
# Purchasing Analysis (Age)
# New column to hold Age groupings. Then group by bins
purchase_data["Age Group"]=pd.cut(purchase_data["Age"],bin,labels=bin_ages)
purch_age_grouped=purchase_data.groupby('Age Group')

In [17]:
# Complete calculations on grouped data set
# purchase count
purch_count=purch_age_grouped['SN'].count()

# avg. purchase price
avg_purch_price = (purch_age_grouped['Price'].mean()).map("${:,.2f}".format)
avg_purch_price

# Total purchase value
tot_purch_value = (purch_age_grouped['Price'].sum()).map("${:,.2f}".format)

# avg. purchase total per person
avg_total_purch = (purch_age_grouped['Price'].sum()/purch_age_grouped['SN'].nunique()).map("${:,.2f}".format)

In [18]:
# Create summary DataSet to present 'Purchasing Analysis (Age)'
Purch_analysis_age=pd.DataFrame({'Purchase Count':purch_count,
                                 'Average Purchase Price': avg_purch_price,
                                 'Total Purchase Value':tot_purch_value,
                                 'Avg Total Purchase per Person': avg_total_purch
                                })

Purch_analysis_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [19]:
# Top Spenders- group by SN
ts_group=purchase_data.groupby('SN')

In [22]:
# Perform calculations on grouped data
# purchase_count
ts_purch_count=ts_group['Purchase ID'].count()

# average purchase price
ts_avg_purch_price=(ts_group['Price'].mean()).map("${:,.2f}".format)

#total purchase price
ts_total_purch_price=ts_group['Price'].sum()

In [23]:
# Make summary df to hold the calculated series
ts_summ = pd.DataFrame({'Purchase Count': ts_purch_count,
                           'Average Purchase Price': ts_avg_purch_price,
                           'Total Purchase Value': ts_total_purch_price
    })
#need to sort column before formatting
ts_summary=ts_summ.sort_values('Total Purchase Value', ascending=False)
#format column
ts_summary['Total Purchase Value']=ts_summary['Total Purchase Value'].map('${:,.2f}'.format)
ts_summary.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [24]:
group_by_id=purchase_data.groupby(["Item ID","Item Name"])

In [25]:
#Most Popular Items
#Retrieve the Item ID, Item Name, and Item Price columns
item_count=pd.DataFrame(group_by_id['Purchase ID'].count())
item_count_rename=item_count.rename(columns={'Purchase ID':"Purchase Count"})

In [26]:
#item price

item_price=pd.DataFrame(group_by_id['Price'].min())

item_price_rename=item_price.rename(columns={"Price":"Item Price"})
item_price_rename

item_price=pd.DataFrame(group_by_id['Price'].min())
item_price_rename=item_price.rename(columns={"Price":"Item Price"})
item_price_format=item_price_rename['Item Price'].map("${:,.2f}".format)

item_price_format=pd.DataFrame(item_price_format)

In [27]:
#total purchase value
item_total=pd.DataFrame(group_by_id['Price'].sum())
item_total_rename=item_total.rename(columns={'Price':"Total Purchase Value"})
item_total_format=pd.DataFrame(item_total_rename["Total Purchase Value"].map("${:,.2f}".format))

In [28]:
popular_summary=pd.merge(
    pd.merge(item_count_rename,item_price_format, left_index=True, right_index=True),
                         item_total_format,left_index=True, right_index=True)

popular_summary=popular_summary.sort_values("Purchase Count", ascending=False)
popular_summary.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [29]:
# Most Profitable Items

In [30]:
# Table is already formatted with $ (obj), table needs to be sorted prior to formatting of "total purcahse value"
# use item_total_rename instead of formated dataframe in merge
profitable_item=pd.merge(
    pd.merge(item_count_rename,item_price_format, left_index=True, right_index=True),
                         item_total_rename,left_index=True, right_index=True)

In [31]:
#sort table by total purchase
profitable_sort=profitable_item.sort_values('Total Purchase Value', ascending = False)

In [32]:
#format total purchase value column
profitable_sort['Total Purchase Value'] = profitable_sort['Total Purchase Value'].map('${:,.2f}'.format)

In [33]:
profitable_sort.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
